# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [1]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('./emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('./emails/ham', 'ham'))


Let's have a look at that DataFrame:

In [4]:
data.head()

,class,message
./emails/spam/00097.013347cc91e7d0915074dccb0428883f,spam,=================================\n\n\n\nGuara...
./emails/spam/00331.a61788d316e7393c8bbf8ee19b24c713,spam,<HTML><HEAD><TITLE></TITLE><META http-equiv=3D...
./emails/spam/00235.45b5f386cf62b5865d9d4440d8b78aab,spam,NIGERIA ELECTIRC POWER AUTHORITY \n\nFEDERAL S...
./emails/spam/00075.28a918cd03a0ef5aa2f1e0551a798108,spam,Dear Sir / Madam\n\n\n\nIf you are fed up of b...
./emails/spam/00085.f63a9484ac582233db057dbb45dc0eaf,spam,HELLO...By reading the short Summary just belo...


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [5]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [6]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], 
      dtype='|S4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [11]:
test = ['Free viagra']
test_counts = vectorizer.transform(test)
classifier.predict(test_counts)

array(['spam'], 
      dtype='|S4')